# 오차 제곱합

In [19]:
import numpy as np
def sum_squares_error(y, t):
    return 0.5 * np.sum((y-t)**2)
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0 ,0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
print(sum_squares_error(np.array(y), np.array(t)))
#잘못된 자료
wy = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0 ,0.0]
print(sum_squares_error(np.array(wy), np.array(t)))

0.09750000000000003
0.2725


In [20]:
#교차 엔트로피 오차(cross entropy error)
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0 ,0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
print(cross_entropy_error(np.array(y), np.array(t)))
wy = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0 ,0.0]
print( cross_entropy_error(np.array(wy), np.array(t)))

0.510825457099338
0.510825457099338


In [1]:
from functions import *
from gradient import numerical_gradient

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01): #입력 크기, 히든 레이어 크기, 출력 크기, 실수값으로 조지기
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) #입력x히든 사이즈로 난수 배열 만들고 실수로 값 변환
        self.params['b1'] = np.zeros(hidden_size)   #히든 레이어 개수만큼 
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = identity_function(a2)
        return y
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        accuracy = np.sum(y == t) / float(x.shape[0]) #np.sum(y==t) y==t인 case의 횟수를 저장, 이를 평균을 내줌
        return accuracy                               #그렇게 구한 정확도를 반환
    
    def numerical_gradient(self, x, t):                 #손실함수를 w에 대해 미분함 -> gradient를 반환
        loss_W = lambda W: self.loss(x, t)
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        batch_num = x.shape[0]
        #forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = identity_function(a2)
        #backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)
        return grads

from datasetsub.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

In [3]:
#미니배치 학습
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]
batch_mask

array([19445, 18610,  5946, 51604, 27442, 31262, 37517, 13103, 53043,
        6500], dtype=int32)

# 수치 미분

In [5]:
# 미분
def numerical_diff(f, x):
    h = 1e-4
    return (f(x+h) - f(x-h)) / (2*h)

#수치 미분의 예
def fun1(x):
    return 0.01*x**2 + 0.1*x
print( numerical_diff(fun1, 5) )

print( numerical_diff(fun1, 10) )

0.1999999999990898
0.2999999999986347
